# Word Embeddings : le modèle Word2Vec

## Imports

In [71]:
import sys
import os
import re
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

### telecharger les stops words

In [72]:
# Télécharger stopwords
nltk.download('stopwords')
sw = set(stopwords.words("french"))

# Stopwords supplémentaires
extra_sw = {
    "les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
    "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
    "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
    "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
    "van", "het", "autre", "jusqu", "ville", "rossel", "dem", "etxc", "elles", "dés",
    "prixx", "écr", "géné", "app", "adr", "mod", "bur", "trav", "et", "de", "à", "en",
    "le", "la", "du", "des", "aux", "un", "une"
}
sw |= extra_sw

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ilaria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Fonction nettoyage de texte

In [73]:
def clean_text_string(text):
    # Supprimer caractères non alphabétiques
    text = re.sub(r"[^a-zA-ZÀ-ÿ\s]", " ", text)
    # Enlever les répétitions de lettres aberrantes (ex: iiiii, llll)
    text = re.sub(r"(.)\1{2,}", r"\1", text)
    # Minuscules
    text = text.lower()
    # Tokenisation simple
    words = nltk.wordpunct_tokenize(text)
    # Filtrage avancé
    kept = [
        w for w in words
        if (
            len(w) > 2 and           # minimum 3 lettres
            len(w) < 20 and          # éviter les tokens trop longs ("ltshdyyysnkjdlqj")
            w.isalpha() and
            w not in sw and
            not w.startswith(("ij", "vv", "qq")) and  # motifs OCR fréquents
            not re.match(r".*[0-9].*", w)             # pas de chiffres
        )
    ]
    return " ".join(kept)


### Lecture du texte nettoyé

In [74]:
# Chemin vers ton fichier
infile = "../../data/sents.txt"

# Lecture de toutes les lignes
with open(infile, encoding="utf-8") as f:
    texts = f.readlines()

# Nettoyage
cleaned_texts = [clean_text_string(text) for text in texts]
# On enlève les lignes vides après nettoyage
cleaned_texts = [t for t in cleaned_texts if t.strip() != ""]

print(f"Nombre de phrases nettoyées : {len(cleaned_texts)}")
print(cleaned_texts[:5])  # afficher les 5 premières phrases pour vérification


Nombre de phrases nettoyées : 891576
['imnri hmu marché tenu hors villa', 'vaain téicj races indigènes', 'rasa îichakdui taureaux iallsènas', 'hollandais dufr', 'îdto vachei laitières vante']


## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [75]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [76]:
infile = f"../../data/sents.txt"
sentences = MySentences(infile)

In [77]:
import os
print(os.getcwd())


/Users/ilaria/Desktop/STIC/traitement_auto_corpus/tac/tps/tp3


### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [78]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [79]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [80]:
len(bigram_phrases.vocab.keys())

4310706

Prenons une clé au hasard :

In [81]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [82]:
bigram_phrases.vocab[key_]

42

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [83]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [84]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [85]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [86]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [87]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races', 'indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 

## Entrainement d'un modèle Word2Vec sur ce corpus

In [88]:
# Liste de combinaisons de paramètres à tester
params_list = [
    {"vector_size": 50, "window": 5, "min_count": 5},
    {"vector_size": 100, "window": 10, "min_count": 5},
    {"vector_size": 100, "window": 5, "min_count": 10},
]

models = {}

for i, params in enumerate(params_list):
    print(f"\n Entraînement du modèle {i+1} avec paramètres : {params}")
    model = Word2Vec(
        corpus,
        vector_size=params["vector_size"],
        window=params["window"],
        min_count=params["min_count"],
        workers=4,
        epochs=5
    )
    models[f"model_{i+1}"] = model
    model.save(f"../../data/word2vec_model_{i+1}.model")
    print(f" Modèle {i+1} sauvegardé !")



 Entraînement du modèle 1 avec paramètres : {'vector_size': 50, 'window': 5, 'min_count': 5}


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


 Modèle 1 sauvegardé !

 Entraînement du modèle 2 avec paramètres : {'vector_size': 100, 'window': 10, 'min_count': 5}


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


 Modèle 2 sauvegardé !

 Entraînement du modèle 3 avec paramètres : {'vector_size': 100, 'window': 5, 'min_count': 10}


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


 Modèle 3 sauvegardé !


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


## Exploration et comparaison des modèles

In [90]:
# Charger et explorer les modèles entraînés
for name, model in models.items():
    print(f"\n=== Exploration de {name} ===")

    # 3 exemples de similarité
    pairs = [("ministre", "roi"), ("homme", "femme"), ("paris", "londres")]
    for w1, w2 in pairs:
        try:
            sim = model.wv.similarity(w1, w2)
            print(f"Similarité({w1}, {w2}) = {sim:.3f}")
        except KeyError:
            print(f"{w1} ou {w2} absent du vocabulaire.")

    # 3 exemples de recherche de mots proches
    terms = ["ministre", "guerre", "belgique"]
    for t in terms:
        try:
            print(f"\nMots les plus proches de '{t}' :")
            for word, score in model.wv.most_similar(t, topn=5):
                print(f"  {word:20s} {score:.3f}")
        except KeyError:
            print(f"{t} absent du vocabulaire.")



=== Exploration de model_1 ===
Similarité(ministre, roi) = 0.487
Similarité(homme, femme) = 0.628
Similarité(paris, londres) = 0.866

Mots les plus proches de 'ministre' :
  secretaire           0.881
  president            0.848
  adjoint              0.837
  en_remplacement      0.816
  senateur             0.813

Mots les plus proches de 'guerre' :
  la_compagnie         0.811
  retraite             0.799
  la_defense           0.789
  pekin                0.787
  la_creation          0.777

Mots les plus proches de 'belgique' :
  la_banque            0.783
  suisse               0.778
  la_societe           0.762
  nationale            0.753
  hollande             0.749

=== Exploration de model_2 ===
Similarité(ministre, roi) = 0.471
Similarité(homme, femme) = 0.547
Similarité(paris, londres) = 0.792

Mots les plus proches de 'ministre' :
  secretaire           0.841
  president            0.808
  au_ministere         0.800
  ancien_ministre      0.774
  vice_-_president     0.76

#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [91]:
outfile = f"../../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [92]:
model = Word2Vec.load("../../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [93]:
model.wv["ministre"]
#moins c'est moins proche, plus c'est plur proche

array([ 1.7086865 ,  1.2003068 , -0.10184547, -0.1903843 , -1.234053  ,
       -0.7660085 ,  3.020541  ,  0.766255  , -0.9248277 ,  0.5010514 ,
        1.9193925 , -1.3724992 ,  0.40011278,  0.8110669 , -0.13830736,
       -1.426515  ,  1.2293569 ,  2.0312908 , -1.6526984 , -0.36108395,
        0.34077248,  0.4256754 ,  0.5544796 , -0.1204455 ,  0.21134059,
       -0.28636232, -0.4802511 ,  0.3701363 ,  0.50864494,  1.7301176 ,
       -1.2834626 ,  0.5587825 ,  0.09995482,  0.6984777 , -0.41349372,
        0.18058398, -1.2498895 , -0.8845742 , -0.9236684 ,  0.8656235 ,
        0.62683547, -1.4011663 ,  1.5751262 , -1.1922377 , -0.30823973,
       -0.4632091 , -0.29010653, -1.1167163 , -0.5207948 ,  0.90157586,
       -0.09536862,  0.05685811, -2.1603014 ,  0.799923  , -2.0085733 ,
       -0.6032597 , -1.6304982 ,  0.32362062, -0.31175053, -0.56017685,
        1.6212375 ,  0.4235438 ,  0.09802127,  1.8521875 , -0.2896857 ,
       -2.747034  ,  0.27558044,  1.795015  , -1.1788349 , -1.63

### Calculer la similarité entre deux termes

In [94]:
model.wv.similarity("ministre", "roi")

np.float32(0.4689652)

### Chercher les mots les plus proches d'un terme donné

In [95]:
model.wv.most_similar("ministre", topn=10)

[('secretaire', 0.82060706615448),
 ('president', 0.7828721404075623),
 ('membre', 0.7756786346435547),
 ('vice_-_president', 0.7555203437805176),
 ('adjoint', 0.7521445751190186),
 ('bourgmestre', 0.7444619536399841),
 ('depute', 0.7438755631446838),
 ('conseiller', 0.7424141764640808),
 ('president_du_conseil', 0.7396537065505981),
 ('senateur', 0.7392389178276062)]

### Faire des recherches complexes à travers l'espace vectoriel

In [96]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))
#on retire le concept belgique aux concept paris et londre, 

[('marseille', 0.7002578377723694), ('la_monnaie', 0.6806557774543762), ('berlin', 0.6692238450050354), ('new_-_york', 0.6341982483863831), ('ce_matin', 0.6011483669281006), ('au_theatre', 0.5973613262176514), ('washington', 0.5830581784248352), ('nice', 0.5765511393547058), ('dimanche', 0.564723789691925), ('toulon', 0.5599889159202576)]
